In [ ]:
import pandas as pd
import numpy as np
import os

# --- Constants from your Algorithm ---
LAMBDAS = [0.5, 0.757858283, 0.870550563, 0.933032992, 0.965936329, 0.982820599]
NFS = [1.0000, 1.0000, 1.0000, 1.0000, 1.0020, 1.0462]
WINDOW_SIZE = 180
FILE_PATH = "/Users/frederickmarvel/GodlHodl/data/btcusd_2015_2026.csv"

def calculate_ewma(window_prices, lam, nf):
    """Calculates EWMA based on your specific logic."""
    # Reverse so index 0 is the most recent price
    rev_prices = window_prices[::-1]
    
    weights = np.array([(1 - lam) * (lam ** i) for i in range(WINDOW_SIZE)])
    normalization_factor = 1 / np.sum(weights)
    weighted_sum = np.sum(weights * rev_prices * nf)
    
    return normalization_factor * weighted_sum

def run_trend_analysis(input_path):
    if not os.path.exists(input_path):
        print(f"Error: File not found at {input_path}")
        return

    # 1. Load Data with delimiter detection
    print("Loading dataset...")
    # Using sep=None engine='python' tells pandas to guess if it's , or ;
    df = pd.read_csv(input_path, sep=None, engine='python', quotechar='"')

    # 2. Clean Column Names
    # This removes extra spaces or quotes that cause KeyError
    df.columns = df.columns.str.replace('"', '').str.strip()
    print(f"Detected columns: {list(df.columns)}")

    # 3. Preprocess Indonesian Format
    # Check if 'Terakhir' exists after cleaning
    target_col = 'Terakhir'
    if target_col not in df.columns:
        print(f"Error: Could not find '{target_col}' column. Available: {list(df.columns)}")
        return

    # Convert to string first to handle any mixed types, then clean format
    df['Close'] = (df[target_col]
                   .astype(str)
                   .str.replace('.', '', regex=False)
                   .str.replace(',', '.', regex=False)
                   .astype(float))
    
    # Convert Date and sort Ascending (Oldest -> Newest)
    df['Tanggal'] = pd.to_datetime(df['Tanggal'], dayfirst=True)
    df = df.sort_values('Tanggal').reset_index(drop=True)

    print(f"Dataset ready: {len(df)} rows from {df['Tanggal'].min().date()} to {df['Tanggal'].max().date()}")

    # 4. Sliding Window Calculation
    history = []
    
    # Range starts at WINDOW_SIZE so we have enough data for the first calculation
    for i in range(WINDOW_SIZE, len(df) + 1):
        # Extract the 180-day window
        window = df['Close'].iloc[i-WINDOW_SIZE : i].values
        
        # Calculate the 6 MAs
        ma1    = calculate_ewma(window, LAMBDAS[0], NFS[0])
        ma2_5  = calculate_ewma(window, LAMBDAS[1], NFS[1])
        ma5    = calculate_ewma(window, LAMBDAS[2], NFS[2])
        ma10   = calculate_ewma(window, LAMBDAS[3], NFS[3])
        ma20   = calculate_ewma(window, LAMBDAS[4], NFS[4])
        ma40   = calculate_ewma(window, LAMBDAS[5], NFS[5])

        # Indicator logic
        indicators = [
            (ma1 - ma5),
            (ma2_5 - ma10),
            (ma5 - ma20),
            (ma10 - ma40),
        ]
        
        trend_score = sum(1 if x >= 0 else -1 for x in indicators) / 4
        
        history.append({
            'Date': df['Tanggal'].iloc[i-1],
            'Price': df['Close'].iloc[i-1],
            'Trend_Indicator': trend_score
        })

    # 5. Convert to DataFrame
    result_df = pd.DataFrame(history)
    return result_df

# --- Execute ---
if __name__ == "__main__":
    results = run_trend_analysis(FILE_PATH)
    
    if results is not None:
        output_file = "btc_trend_results.csv"
        results.to_csv(output_file, index=False)
        print(f"Analysis complete. Results saved to {output_file}")
        
        print("\nLatest Trend Data:")
        print(results.tail(10))

Loading dataset...
Detected columns: ['price_date', 'price_value_price', 'price_value_yield', 'btc_close']
Error: Could not find 'Terakhir' column. Available: ['price_date', 'price_value_price', 'price_value_yield', 'btc_close']
